In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colorbar
import seaborn as sns

# Third-party libraries
import camb
import pyhmcode as hmcode

# Seabornify
sns.set_theme(style='ticks')

# CAMB verbosity level
camb.set_feedback_level(0)

In [ ]:
# Cosmological parameters
h = 0.7
omc = 0.25
omb = 0.05
mnu = 0.0
w = -1.0
wa = 0.0
ns = 0.97
As = 2.1e-9

# Redshift
z = 0.

# Wavenumber [h/Mpc]
k_max = 20.

In [ ]:
# Use CAMB to get the linear power spectrum

# Get linear power spectrum from CAMB
p = camb.CAMBparams(WantTransfer=True, 
                    WantCls=False, 
                    Want_CMB_lensing=False, 
                    DoLensing=False,
                    NonLinearModel=camb.nonlinear.Halofit(halofit_version='mead2020'),
                   )
p.set_cosmology(H0=h*100., omch2=omc*h**2, ombh2=omb*h**2, mnu=mnu)
p.set_dark_energy(w=w, wa=wa)
p.set_initial_power(camb.InitialPowerLaw(As=As, ns=ns))
p.set_matter_power(redshifts=[z], kmax=k_max, nonlinear=True)

# Compute CAMB results
r = camb.get_results(p)
ks, zs, Pk_lin = r.get_linear_matter_power_spectrum(nonlinear=False)

In [ ]:
# HMcode stuff

# Need sigma_8 from CAMB as this is HMcode internal parameter
if z == 0.:
    sigma8 = r.get_sigma8()[0]
else:
    raise ValueError('Error, need z=0 for sigma8')
    
# Need these Omegas
# Note that the calculation of Omega_v is peculiar, but ensures flatness in HMcode
# (very) small differences between CAMB and HMcode otherwise
omv = r.omega_de+r.get_Omega("photon")+r.get_Omega("neutrino") 
omm = p.omegam

# Setup HMcode internal cosmology
c = hmcode.Cosmology()

# Set HMcode internal cosmological parameters
c.om_m = omm
c.om_b = omb
c.om_v = omv
c.h = h
c.ns = ns
c.sig8 = sigma8
c.m_nu = mnu

# Set the linear power spectrum for HMcode
c.set_linear_power_spectrum(ks, zs, Pk_lin)

# halo model calculation; hmcode.HMx2020_matter_w_temp_scaling, hmcode.HMx2020_matter_pressure_w_temp_scaling
mode = hmcode.HMx2020_matter_w_temp_scaling
hmod = hmcode.Halomodel(mode, verbose=False)
fields = [hmcode.field_matter, hmcode.field_cdm, hmcode.field_gas, hmcode.field_stars]
Pk_hm = hmcode.calculate_nonlinear_power_spectrum(c, hmod, fields, verbose=False)

In [ ]:
# Plots

# Parameters
Pkmin = 1e-1; Pkmax = 1e5
alp = np.linspace(1., 0.25, len(zs))

# Initialise
plt.subplots(3, 1, figsize=(10, 10), sharex=True)

labels = ['linear', 'matter-matter', 'CDM-CDM', 'gas-gas', 'stars-stars']
colours = ['black', 'C0', 'C1', 'C2', 'C3']

# P(k)
plt.subplot(3, 1, 1)
for iz, z in enumerate(zs):
    if iz == 0:
        labs = labels
    else:
        labs = 3*[None]
    Pks = [Pk_lin[iz], Pk_hm[0, 0, iz], Pk_hm[1, 1, iz], Pk_hm[2, 2, iz], Pk_hm[3, 3, iz]]
    for (Pk, col, lab) in zip(Pks, colours, labs):
        plt.loglog(ks, Pk, color=col, alpha=alp[iz], label=lab)
plt.xticks([])
plt.ylim((Pkmin, Pkmax))
plt.ylabel(r'$P(k)\,/\,(h^{-1}\mathrm{Mpc})^3$')
plt.legend()

# Ratio to linear theory
plt.subplot(3, 1, 2)
for iz, _ in enumerate(zs):
    Pks = [Pk_lin[iz], Pk_hm[0, 0, iz], Pk_hm[1, 1, iz], Pk_hm[2, 2, iz], Pk_hm[3, 3, iz]]
    for (Pk, col) in zip(Pks, colours):
        plt.loglog(ks, Pk/Pk_lin[iz], color=col, alpha=alp[iz])
plt.xticks([])
plt.ylabel(r'$P(k)\,/\,P^\mathrm{lin}(k)$')

# Ratio to CAMB non-linear (whatever that is)
plt.subplot(3, 1, 3)
for iz, _ in enumerate(zs):
    Pks = [Pk_lin[iz], Pk_hm[0, 0, iz], Pk_hm[1, 1, iz], Pk_hm[2, 2, iz], Pk_hm[3, 3, iz]]
    for (Pk, col) in zip(Pks, colours):
        plt.loglog(ks, Pk/Pk_hm[0, 0, iz], color=col, alpha=alp[iz])
plt.xlabel(r'$k\,/\,h\mathrm{Mpc}^{-1}$')
plt.ylabel(r'$P(k)\,/\,P^\mathrm{nl, CAMB}(k)$')

# Display
plt.show()